Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Automated Machine Learning
_**Orange Juice Sales Forecasting**_

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)

## Introduction
In this example, we use AutoML to find and tune a time-series forecasting model.

Make sure you have executed the [configuration notebook](../../../configuration.ipynb) before running this notebook.

In this notebook, you will:
1. Create an Experiment in an existing Workspace
2. Instantiate an AutoMLConfig 
3. Find and train a forecasting model using local compute
4. Evaluate the performance of the model

The examples in the follow code samples use the [University of Chicago's Dominick's Finer Foods dataset](https://research.chicagobooth.edu/kilts/marketing-databases/dominicks) to forecast orange juice sales. Dominick's was a grocery chain in the Chicago metropolitan area.

## Setup

As part of the setup you have already created a <b>Workspace</b>. To run AutoML, you also need to create an <b>Experiment</b>. An Experiment is a named object in a Workspace which represents a predictive task, the output of which is a trained model and a set of evaluation metrics for the model. 

In [1]:
import azureml.core
import pandas as pd
import numpy as np
import logging
import warnings
# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None


from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
ws = Workspace.from_config()

# choose a name for the run history container in the workspace
experiment_name = 'automl-ojsalesforecasting'
# project folder
project_folder = './sample_projects/automl-local-ojsalesforecasting'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

Found the config file in: /home/nbuser/library/Module_2_Get Started with AzureML/config.json


,
Location,eastus
Project Directory,./sample_projects/automl-local-ojsalesforecasting
Resource Group,aiconfnyc
Run History Name,automl-ojsalesforecasting
SDK version,1.0.17
Subscription ID,5d9f0467-1e93-45cb-9531-b2f8e715feb1
Workspace,aiconfnycws


## Data
You are now ready to load the historical orange juice sales data. We will load the CSV file into a plain pandas DataFrame; the time column in the CSV is called _WeekStarting_, so it will be specially parsed into the datetime type.

In [3]:
time_column_name = 'WeekStarting'
data = pd.read_csv("dominicks_OJ.csv", parse_dates=[time_column_name])
data.head()

,WeekStarting,Store,Brand,Quantity,logQuantity,Advert,Price,Age60,COLLEGE,INCOME,Hincome150,Large HH,Minorities,WorkingWoman,SSTRDIST,SSTRVOL,CPDIST5,CPWVOL5
0,1990-06-14,2,dominicks,10560,9.26,1,1.59,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
1,1990-06-14,2,minute.maid,4480,8.41,0,3.17,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
2,1990-06-14,2,tropicana,8256,9.02,0,3.87,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
3,1990-06-14,5,dominicks,1792,7.49,1,1.59,0.12,0.32,10.92,0.54,0.10,0.05,0.41,3.80,0.68,1.60,0.74
4,1990-06-14,5,minute.maid,4224,8.35,0,2.99,0.12,0.32,10.92,0.54,0.10,0.05,0.41,3.80,0.68,1.60,0.74


Each row in the DataFrame holds a quantity of weekly sales for an OJ brand at a single store. The data also includes the sales price, a flag indicating if the OJ brand was advertised in the store that week, and some customer demographic information based on the store location. For historical reasons, the data also include the logarithm of the sales quantity. The Dominick's grocery data is commonly used to illustrate econometric modeling techniques where logarithms of quantities are generally preferred.    

The task is now to build a time-series model for the _Quantity_ column. It is important to note that this dataset is comprised of many individual time-series - one for each unique combination of _Store_ and _Brand_. To distinguish the individual time-series, we thus define the **grain** - the columns whose values determine the boundaries between time-series: 

In [4]:
grain_column_names = ['Store', 'Brand']
nseries = data.groupby(grain_column_names).ngroups
print('Data contains {0} individual time-series.'.format(nseries))

Data contains 249 individual time-series.


### Data Splitting
For the purposes of demonstration and later forecast evaluation, we now split the data into a training and a testing set. The test set will contain the final 20 weeks of observed sales for each time-series.

In [5]:
ntest_periods = 20

def split_last_n_by_grain(df, n):
    """
    Group df by grain and split on last n rows for each group
    """
    df_grouped = (df.sort_values(time_column_name) # Sort by ascending time
                  .groupby(grain_column_names, group_keys=False))
    df_head = df_grouped.apply(lambda dfg: dfg.iloc[:-n])
    df_tail = df_grouped.apply(lambda dfg: dfg.iloc[-n:])
    return df_head, df_tail

X_train, X_test = split_last_n_by_grain(data, ntest_periods)

## Modeling

For forecasting tasks, AutoML uses pre-processing and estimation steps that are specific to time-series. AutoML will undertake the following pre-processing steps:
* Detect time-series sample frequency (e.g. hourly, daily, weekly) and create new records for absent time points to make the series regular. A regular time series has a well-defined frequency and has a value at every sample point in a contiguous time span 
* Impute missing values in the target (via forward-fill) and feature columns (using median column values) 
* Create grain-based features to enable fixed effects across different series
* Create time-based features to assist in learning seasonal patterns
* Encode categorical variables to numeric quantities

AutoML will currently train a single, regression-type model across **all** time-series in a given training set. This allows the model to generalize across related series.

You are almost ready to start an AutoML training job. We will first need to create a validation set from the existing training set (i.e. for hyper-parameter tuning): 

In [7]:
nvalidation_periods = 20
X_train, X_validate = split_last_n_by_grain(X_train, nvalidation_periods)

We also need to separate the target column from the rest of the DataFrame: 

In [8]:
target_column_name = 'Quantity'
y_train = X_train.pop(target_column_name).values
y_validate = X_validate.pop(target_column_name).values 

## Train

The AutoMLConfig object defines the settings and data for an AutoML training job. Here, we set necessary inputs like the task type, the number of AutoML iterations to try, and the training and validation data. 

For forecasting tasks, there are some additional parameters that can be set: the name of the column holding the date/time and the grain column names. A time column is required for forecasting, while the grain is optional. If a grain is not given, the forecaster assumes that the whole dataset is a single time-series. We also pass a list of columns to drop prior to modeling. The _logQuantity_ column is completely correlated with the target quantity, so it must be removed to prevent a target leak. 

|Property|Description|
|-|-|
|**task**|forecasting|
|**primary_metric**|This is the metric that you want to optimize.<br> Forecasting supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>
|**iterations**|Number of iterations. In each iteration, Auto ML trains a specific pipeline on the given data|
|**X**|Training matrix of features, shape = [n_training_samples, n_features]|
|**y**|Target values, shape = [n_training_samples, ]|
|**X_valid**|Validation matrix of features, shape = [n_validation_samples, n_features]|
|**y_valid**|Target values for validation, shape = [n_validation_samples, ]
|**enable_ensembling**|Allow AutoML to create ensembles of the best performing models
|**debug_log**|Log file path for writing debugging information
|**path**|Relative path to the project folder.  AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder. 

In [9]:
automl_settings = {
    'time_column_name': time_column_name,
    'grain_column_names': grain_column_names,
    'drop_column_names': ['logQuantity']
}

automl_config = AutoMLConfig(task='forecasting',
                             debug_log='automl_oj_sales_errors.log',
                             primary_metric='normalized_root_mean_squared_error',
                             iterations=10,
                             X=X_train,
                             y=y_train,
                             X_valid=X_validate,
                             y_valid=y_validate,
                             enable_ensembling=False,
                             path=project_folder,
                             verbosity=logging.INFO,
                            **automl_settings)

You can now submit a new training run. For local runs, the execution is synchronous. Depending on the data and number of iterations this operation may take several minutes.
Information from each iteration will be printed to the console.

In [10]:
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_1c5c59c3-cad5-440a-92ba-adbba5d6b846
********************************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
SAMPLING %: Percent of the training data to sample.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
********************************************************************************************************************

 ITERATION   PIPELINE                                       SAMPLING %  DURATION      METRIC      BEST
         0   StandardScalerWrapper ElasticNet               100.0000    0:00:34       0.0305    0.0305
         1   StandardScalerWrapper ElasticNet               100.0000    0:00:25       0.0300    0.0300
         2   StandardScalerWrapper ElasticNet               100

In [11]:
local_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-ojsalesforecasting,AutoML_1c5c59c3-cad5-440a-92ba-adbba5d6b846,automl,Completed,Link to Azure Portal,Link to Documentation


### Retrieve the Best Model
Each run within an Experiment stores serialized (i.e. pickled) pipelines from the AutoML iterations. We can now retrieve the pipeline with the best performance on the validation dataset:

In [12]:
best_run, fitted_pipeline = local_run.get_output()
fitted_pipeline.steps

[('timeseriestransformer', TimeSeriesTransformer(logger=None)),
 ('standardscalerwrapper',
  <automl.client.core.common.model_wrappers.StandardScalerWrapper at 0x7fb1a48aeac8>),
 ('lightgbmregressor',
  <automl.client.core.common.model_wrappers.LightGBMRegressor at 0x7fb1a48ae6a0>)]

### Make Predictions from the Best Fitted Model
Now that we have retrieved the best pipeline/model, it can be used to make predictions on test data. First, we remove the target values from the test set:

In [13]:
y_test = X_test.pop(target_column_name).values

In [14]:
X_test.head()

,WeekStarting,Store,Brand,logQuantity,Advert,Price,Age60,COLLEGE,INCOME,Hincome150,Large HH,Minorities,WorkingWoman,SSTRDIST,SSTRVOL,CPDIST5,CPWVOL5
24192,1992-05-21,2,dominicks,9.18,0,1.69,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
24441,1992-05-28,2,dominicks,10.73,0,1.69,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
24675,1992-06-04,2,dominicks,9.95,0,1.74,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
24909,1992-06-11,2,dominicks,8.79,0,2.09,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
25152,1992-06-18,2,dominicks,8.52,0,2.05,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38


To produce predictions on the test set, we need to know the feature values at all dates in the test set. This requirement is somewhat reasonable for the OJ sales data since the features mainly consist of price, which is usually set in advance, and customer demographics which are approximately constant for each store over the 20 week forecast horizon in the testing data. 

The target predictions can be retrieved by calling the `predict` method on the best model:

In [15]:
y_pred = fitted_pipeline.predict(X_test)

### Calculate evaluation metrics for the prediction
To evaluate the accuracy of the forecast, we'll compare against the actual sales quantities for some select metrics, included the mean absolute percentage error (MAPE).

In [16]:
def MAPE(actual, pred):
    """
    Calculate mean absolute percentage error.
    Remove NA and values where actual is close to zero
    """
    not_na = ~(np.isnan(actual) | np.isnan(pred))
    not_zero = ~np.isclose(actual, 0.0)
    actual_safe = actual[not_na & not_zero]
    pred_safe = pred[not_na & not_zero]
    APE = 100*np.abs((actual_safe - pred_safe)/actual_safe)
    return np.mean(APE)

print("[Test Data] \nRoot Mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print('mean_absolute_error score: %.2f' % mean_absolute_error(y_test, y_pred))
print('MAPE: %.2f' % MAPE(y_test, y_pred))

[Test Data] 
Root Mean squared error: 16158.98
mean_absolute_error score: 8588.16
MAPE: 73.83
